### 포켓몬 페이지에서 이미지 데이터 수집!
- 폴더 생성
- 이미지 링크 수집 후, 다운로드 자동화 (바탕화면에 저장)

In [2]:
# 운영체제를 다룰 수 있는 라이브러리
import os # 폴더 생성, 파일 삭제 등 운영체제에 접근하여 관리할 수 있는 라이브러리

In [16]:
# 폴더 생성해보기 -> 바탕화면
# os.mkdir("C:\\Users\\smhrd1\\Desktop\\포켓몬") # 이스케이프
# os.mkdir("C:/Users/smhrd1/Desktop/포켓몬") # 슬래쉬 활용

# 이미 생성된 경우, 중복된 폴더명으로 생성할 수 없음!

In [22]:
# 해당 경로에 폴더가 있는지 확인하기
if os.path.isdir("C:\\Users\\smhrd1\\Desktop\\포켓몬") :
    print("폴더가 이미 존재함")
else : 
    os.mkdir("C:\\Users\\smhrd1\\Desktop\\포켓몬")
    print("폴더 생성 완료!")

폴더가 이미 존재함


In [26]:
from selenium import webdriver as wb # 테스트 브라우저 관리 도구
from selenium.webdriver.common.by import By  # 경로 지정 방식
from selenium.webdriver.common.keys import Keys  # 키보드 입력값

from bs4 import BeautifulSoup as bs # 파싱도구

In [28]:
driver = wb.Chrome()
url = "https://www.pokemonkorea.co.kr/pokedex#pokedex_1"
driver.get(url)

### 요소 중 속성 추출
- soup : 인덱싱으로 접근
- selenium : get_attribute()

In [ ]:
soup = bs(driver.page_source, "lxml")

imgs = soup.select("img.img-fluid")

In [50]:
imgs[0].text # 쌍태그안에 content를 가져올 때 사용되는 속성
imgs[0]["src"] # soup에서는 인덱싱으로 요소 속성 접근 가능!

'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000101.png'

```python
# selenium
driver.find_elements(By.CSS_SELECTOR, "img.img-fluid")[0].get_attribute("src")
```

In [57]:
len(imgs)
# 길이가 한참 모자람 (총 이미지 개수는 약 1000개가량)
# 현재 로드되어 있는 이미지만 수집된 것을 확인!

18

### 스크롤 내리기
- 효율적으로 내리기 위해 창 최대화!

In [62]:
# 창 최대화
driver.maximize_window()

In [66]:
# END, Page Down, Space bar, 방향키(아래)
# END가 가장 효율적!
body = driver.find_element(By.TAG_NAME, "body")

In [98]:
body.send_keys(Keys.END)

In [102]:
# 페이지 길이 가져오기!
driver.execute_script("return document.body.scrollHeight") # 스크롤 길이 확인
# 비동기로 인해 길이값이 제대로 업데이트가 이루어지지 않는 상황도 발생!!
# 그래서 좀 더 쉽게 비교하는 방법!!

3267

In [108]:
import time

In [110]:
# 페이지 스크롤 알고리즘! 
while True : 
    old = driver.page_source # 기존 페이지 소스

    # 스크롤 하강
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.END)
    time.sleep(2)
    
    new = driver.page_source # 갱신된 페이지 소스

    if old == new : # 같은지 확인
        print("스크롤 하강 완료")
        break
    else : 
        pass

스크롤 하강 완료


In [112]:
# 요소가 완전히 로드 된 상태에서 img 태그 접근
soup = bs(driver.page_source, "lxml")
imgs = soup.select("img.img-fluid")

In [114]:
len(imgs)

1252

In [126]:
type(imgs[1])

bs4.element.Tag

```python
img_link = []

for img in imgs : 
    img_link.append(img["src"])
```

In [139]:
# 반복문 진핸현황 출력!
from tqdm import tqdm # 반복문 현황 출력하는 도구

for img in tqdm(imgs) : 
    img_link.append(img["src"])

100%|████████████████████████████████████████████████████████| 1252/1252 [00:00<00:00, 627541.66it/s]


In [128]:
# 링크만 수집하기!
img_link = [ img["src"] for img in imgs ]

In [130]:
img_link

['https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000101.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000201.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000301.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000302.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000303.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000401.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000501.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000601.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000602.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000603.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000604.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000701.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000801.png',
 'https://data1.pokemonkorea.co.kr/newdata/pokedex/mid/000901.png',
 'https://data1.pokemonkorea.co.kr/newdata/poked

### url을 활용하여 이미지 다운로드!

In [134]:
from urllib.request import urlretrieve # url 기반으로 다운로드 할 수 있는 도구!

In [136]:
# 1개만 test!
# urlretrieve("이미지 링크", "경로 및 파일명.확장자")
urlretrieve(img_link[0], "C:\\Users\\smhrd1\\Desktop\\포켓몬\\포켓몬1.jpg")

('C:\\Users\\smhrd1\\Desktop\\포켓몬\\포켓몬1.jpg',
 <http.client.HTTPMessage at 0x217d0cbc6b0>)

In [142]:
driver.quit()

### 실습! 하나의 셀에 코드 합치기!
1. 폴더 생성 자동화
2. 스크롤 하강
3. 링크 수집
4. 링크 기반으로 다운로드 -> tqdm 활용!
5. 자원반납

In [148]:
# 1. 폴더 생성 자동화
# 해당 경로에 폴더가 있는지 확인하기
if os.path.isdir("C:\\Users\\smhrd1\\Desktop\\포켓몬") :
    print("폴더가 이미 존재함")
else : 
    os.mkdir("C:\\Users\\smhrd1\\Desktop\\포켓몬")
    print("폴더 생성 완료!")

driver = wb.Chrome()
url = "https://www.pokemonkorea.co.kr/pokedex#pokedex_1"
driver.get(url)

time.sleep(2) # 브라우저가 완전히 로드되게끔 대기

driver.maximize_window() # 최대화

# 2. 스크롤 하강
# 페이지 스크롤 알고리즘! 
while True : 
    old = driver.page_source # 기존 페이지 소스

    # 스크롤 하강
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.END)
    time.sleep(1.5)
    
    new = driver.page_source # 갱신된 페이지 소스

    if old == new : # 같은지 확인
        print("스크롤 하강 완료")
        break
    else : 
        pass
    
# 3. 링크 수집
# 요소가 완전히 로드 된 상태에서 img 태그 접근
soup = bs(driver.page_source, "lxml")
imgs = soup.select("img.img-fluid")

# 링크만 수집하기!
img_link = [ img["src"] for img in imgs ]

# 4. 링크 기반으로 다운로드 -> tqdm 활용
for i in tqdm(range(len(img_link))) : 
    urlretrieve(img_link[i], f"C:\\Users\\smhrd1\\Desktop\\포켓몬\\포켓몬{i+1}.jpg")

# 5. 자원반납
driver.quit()

폴더가 이미 존재함
스크롤 하강 완료


100%|████████████████████████████████████████████████████████████| 1252/1252 [00:50<00:00, 24.95it/s]


In [146]:
driver.quit()

```python

# 3. 링크 수집
# 요소가 완전히 로드 된 상태에서 img 태그 접근
soup = bs(driver.page_source, "lxml")
imgs = soup.select("img.img-fluid")

# 링크만 수집하기!
img_link = [ img["src"] for img in imgs ]

###################################################

# selenium
imgs = driver.find_elements(By.CSS_SELECTOR, "img.img-fluid")

img_link = []
for i in range(len(imgs)) : 
    img_link.append(imgs[i].get_attribute("src"))


```